In [17]:
import random

In [18]:
class Sentiment: 
    NEGATIVE = "NEGATIVE"
    NEUTRAL = "NEUTRAL"
    POSITIVE = "POSITIVE"
    
class Review: 
    def __init__(self, text, score):
        self.text = text 
        self.score = score
        self.sentiment = self.get_sentiment()
        
    def get_sentiment(self):
        if self.score <= 2: return Sentiment.NEGATIVE
        elif self.score == 3:  return Sentiment.NEUTRAL
        else: return Sentiment.POSITIVE #Score of 4 or 5

class ReviewContainer: 
    def __init__(self, reviews):
        self.reviews = reviews
    
    def get_text(self):
        return [review.text for review in self.reviews]
    
    def get_sentiment(self):
        return [review.sentiment for review in self.reviews]
    
    def evenly_distribute(self): 
        positive = list(filter(lambda review: review.sentiment == Sentiment.POSITIVE, self.reviews))
        negative = list(filter(lambda review: review.sentiment == Sentiment.NEGATIVE, self.reviews))
        positive_shrunk = positive[:len(negative)]
        self.reviews = negative + positive_shrunk
        random.shuffle(self.reviews)

In [19]:
import json

file_name = './data/books_small_10000.json'
reviews = []

with open(file_name) as f: 
    for line in f:
        review = json.loads(line)
        reviews.append(Review(review['reviewText'], review['overall']))

reviews[5].text 

'I hoped for Mia to have some peace in this book, but her story is so real and raw.  Broken World was so touching and emotional because you go from Mia\'s trauma to her trying to cope.  I love the way the story displays how there is no "just bouncing back" from being sexually assaulted.  Mia showed us how those demons come for you every day and how sometimes they best you. I was so in the moment with Broken World and hurt with Mia because she was surrounded by people but so alone and I understood her feelings.  I found myself wishing I could give her some of my courage and strength or even just to be there for her.  Thank you Lizzy for putting a great character\'s voice on a strong subject and making it so that other peoples story may be heard through Mia\'s.'

In [20]:
from sklearn.model_selection import train_test_split

training, test = train_test_split(reviews, test_size=0.33, random_state=42)
train_container = ReviewContainer(training)
test_container = ReviewContainer(test)

In [21]:
train_container.evenly_distribute()
train_x = train_container.get_text() 
train_y = train_container.get_sentiment()

test_container.evenly_distribute()
test_x = test_container.get_text()
test_y = test_container.get_sentiment()

print(train_y.count(Sentiment.POSITIVE))
print(train_y.count(Sentiment.NEGATIVE))

436
436


Bag of words vecorization   

In [29]:
from sklearn.feature_extraction.text import CountVectorizer, TfidfVectorizer

vectorizer = TfidfVectorizer()
train_x_vectors = vectorizer.fit_transform(train_x)
test_x_vectors = vectorizer.transform(test_x)

print(train_x[0])
print(train_x_vectors[0].toarray())

I found the story interesting and engaging but the writing was highly self conscious.  There were inconsistencies in the story and several times I was left thinking, &#34;yes, and?&#34; The many untied ends and abrupt changes in the narrative make for an unsatisfying read. Love the idea but the execution was mediocre at best.
[[0. 0. 0. ... 0. 0. 0.]]


In [33]:
test_x[0]

"I choose this book because I enjoy books about the 2nd world war.  Great reading and another &#34;can't put the book down.&#34; I reccommend it to all that is into this subject."

In [32]:
# Linear SVM
from sklearn import svm 
clf_svm = svm.SVC(kernel='linear')
clf_svm.fit(train_x_vectors, train_y)
clf_svm.predict(test_x_vectors[0])

I choose this book because I enjoy books about the 2nd world war.  Great reading and another &#34;can't put the book down.&#34; I reccommend it to all that is into this subject.


array(['POSITIVE'], dtype='<U8')

In [31]:
# Decision Tree
from sklearn.tree import DecisionTreeClassifier
clf_dec = DecisionTreeClassifier()
clf_dec.fit(train_x_vectors, train_y)
clf_dec.predict(test_x_vectors[0])

array(['POSITIVE'], dtype='<U8')

In [34]:
# Naive Bayes
from sklearn.naive_bayes import GaussianNB
clf_gnb = DecisionTreeClassifier()
clf_gnb.fit(train_x_vectors, train_y)
clf_gnb.predict(test_x_vectors[0])

array(['POSITIVE'], dtype='<U8')

In [36]:
# Logistic Regression
from sklearn.linear_model import LogisticRegression
clf_log = LogisticRegression()
clf_log.fit(train_x_vectors, train_y)
clf_log.predict(test_x_vectors[0])

array(['POSITIVE'], dtype='<U8')

In [37]:
# Evaluation
print(clf_svm.score(test_x_vectors, test_y))
print(clf_dec.score(test_x_vectors, test_y))
print(clf_gnb.score(test_x_vectors, test_y))
print(clf_log.score(test_x_vectors, test_y))

0.8076923076923077
0.6370192307692307
0.65625
0.8052884615384616


In [38]:
#F1 Scores 
from sklearn.metrics import f1_score
f1_score(test_y, clf_svm.predict(test_x_vectors), average=None, labels=[Sentiment.POSITIVE, Sentiment.NEGATIVE])

array([0.80582524, 0.80952381])

In [39]:
test_set = ['very fun', 'bad book do not buy', 'horrible waste of time']
new_test = vectorizer.transform(test_set)
clf_svm.predict(new_test)

array(['POSITIVE', 'NEGATIVE', 'NEGATIVE'], dtype='<U8')

In [ ]:
# Tuning our model (with Grid Search)
parameters = {'kernel': ('linear', 'rbf'), 'C': (1, 4, 8, 16, 32)}